In [1]:
import os, sys
sys.path.insert(0, os.path.abspath('dolphyn'))
import dolphyn as D
import pandas as pd

In [2]:
protein_seq_file = "./data/PhageScan_pilot_proteinClusterReps.faa" #your protein collection input, our example

prot_file = "dolphyn_peptides.faa"
dna_file_noadap = "dolphyn_oligos_noadap.fasta"
dna_file = "dolphyn_oligos.fasta"

five_prime_adapter = "AGGAATTCCGCTGCGT"
three_prime_adapter = "ATGGTCACAGCTGTGC"

In [3]:
with open(protein_seq_file, 'r') as fp:
    lines = len(fp.readlines())
    print('Total Number of lines:', lines)

Total Number of lines: 6708


In [4]:
def findMostProbableEpitopes(protein_seq_file, epitile_size, testrun = 0, epitope_probability_cutoff = 0.5, training_size=0):
    count = 0
    global_epitopes = {}   
    if training_size==0:
        training_size=epitile_size
    ep, _ = D.initEpiPredictor(training_size)
    
    with open(protein_seq_file, "r") as org:
        for line in org.readlines():
            line = line.strip()
            if testrun!= 0 and count/2 > testrun:
                break
            count = count + 1

            if (count/2)%500 == 0:
                print(str(count/2), "sequences done")

            if line.startswith(">"):
                protein = line[1:]
                print(protein)
            else:
                pseq = line
                protein_length = len(pseq)
                epitope_proba_atpos = {}

                prot_15mer_feat = D.kmer_features_of_protein(pseq,epitile_size)
                probas = [class_proba[1] for class_proba in ep.predict_proba(prot_15mer_feat)]

                for startpos in range(len(pseq)-epitile_size):
                    #epitope_proba_atpos[startpos] = np.min(probas[startpos:startpos+3])
                    #print(startpos,probas[startpos])
                    epitope_proba_atpos[startpos] = probas[startpos]

                max_value = max(probas)
                max_index = probas.index(max_value)
                print(max_index,max_value)
                print(pseq[max_index:max_index+epitile_size])

#                while len(epitope_proba_atpos)>0:
#                    max_epi_proba_pos = max(epitope_proba_atpos, key=epitope_proba_atpos.get) 
#                    min_epi_proba = epitope_proba_atpos[max_epi_proba_pos]
#                    offset = np.argmax(probas[max_epi_proba_pos:max_epi_proba_pos+3])
#                    max_epi_proba_pos+=offset
#                    epi_proba = probas[max_epi_proba_pos]
#                    epi_seq = pseq[max_epi_proba_pos:max_epi_proba_pos+epitile_size]
#                    if min_epi_proba < epitope_probability_cutoff :
#                        break
#                    else:
#                        if epi_seq not in global_epitopes:
#                            global_epitopes[epi_seq] = {"probability":epi_proba,"proteins":[],"start_pos":[]}   
#                        global_epitopes[epi_seq]["proteins"].append(protein)                    
#                        global_epitopes[epi_seq]["start_pos"].append(max_epi_proba_pos)                   
#
#                        for pos in range(max_epi_proba_pos-epitile_size, max_epi_proba_pos+epitile_size):
#                            if pos in epitope_proba_atpos:
#                                del epitope_proba_atpos[pos]
#    return(global_epitopes)

In [5]:
# TAKES TIME
# find the epitopes

import time

start_time = time.time()
# set testrun = 0 to run all! Default is 2
#ge = D.findEpitopes(testrun = 5, protein_seq_file = protein_seq_file, epitile_size=15, epitope_probability_cutoff = 0.5)
findMostProbableEpitopes(testrun = 5, protein_seq_file = protein_seq_file, epitile_size=8,
                              epitope_probability_cutoff = 0.5, training_size=15)

end_time = time.time()

print(str(int(end_time-start_time)) + "s elapsed")

REP_uvig_160801_16_ALL_uvig_80910_1_uvig_80910_13_uvig_160801_16_uvig_213404_1_uvig_213404_13_uvig_225655_4_uvig_225655_17_uvig_359760_4_uvig_359760_18_uvig_379702_1_uvig_379702_13_uvig_512665_1_uvig_578696_11_uvig_578696_25
89 0.76
EAWRIFGS
REP_uvig_80910_2_ALL_uvig_80910_2_uvig_80910_14_uvig_160801_15_uvig_213404_2_uvig_213404_14_uvig_225655_5_uvig_225655_18_uvig_359760_5_uvig_359760_19_uvig_379702_2_uvig_379702_14_uvig_512665_2_uvig_578696_10_uvig_578696_24
57 0.67
LSDGFWYF
REP_uvig_80910_3_ALL_uvig_80910_3_uvig_80910_15_uvig_160801_14_uvig_213404_3_uvig_213404_15_uvig_225655_6_uvig_225655_19_uvig_359760_6_uvig_359760_20_uvig_379702_3_uvig_379702_15_uvig_512665_3_uvig_512665_9_uvig_578696_9_uvig_578696_23
315 0.74
AGKQYQYH
REP_uvig_80910_4_ALL_uvig_80910_4_uvig_80910_16_uvig_160801_8_uvig_160801_13_uvig_213404_4_uvig_213404_16_uvig_225655_7_uvig_225655_20_uvig_359760_7_uvig_359760_21_uvig_379702_4_uvig_379702_16_uvig_512665_4_uvig_512665_10_uvig_578696_8_uvig_578696_22
68 0.67
WVLRT

In [6]:
ep, _ = D.initEpiPredictor(15)
#seq='MAISAYIGIPGSGKSYEAVYNVIIPAFTSGRRVVTNIYGLQKDKITERYPDATGEIIVVDNDDVLKADFFPFKGGEGSFCQFGDLIVIDEAWRIFGSDKDMTAEKKSFIAEHRHFTHPETGISCDLVIVNQSLSNIARFLKDKIETTYRMRKLKALGLNNHYCIDVYSGHKIYKSNLVTSYRNKYNPDIFELYKSYEGNNGNEKQTDKRQSIWNSGKVRFFLVLFPLMFIGSGWLIYSFFSTFGRSDPSPDLTTTDVRDAAMFRSSAATPAPDTPSEPAEPPLSTEWRISGRMTSEGRAFVILVNGAGVLRAVPASSFNYKGMLMSGIIDGERVTLYTGKK'
#seq='IDEAWRIFGSDK'
seq='IDEAWRIFGS'

In [7]:
pd.set_option('display.max_columns', None)
features=D.kmer_features_of_protein(seq,8)
print(features)

   sc_hydrophobic  sc_polaruncharged  sc_poseleccharged  sc_negeleccharged  A  \
0               5                  0                  1                  2  1   
1               5                  0                  1                  2  1   
2               4                  1                  1                  2  1   

   R  N  D  C  Q  E  G  H  I  L  K  M  F  P  S  T  W  Y  V  d_K  d_C  d_G  \
0  1  0  1  0  0  1  0  0  2  0  0  0  1  0  0  0  1  0  0    3    0    0   
1  1  0  1  0  0  1  1  0  2  0  0  0  1  0  0  0  1  0  0    3    0    1   
2  1  0  1  0  0  1  1  0  1  0  0  0  1  0  1  0  1  0  0    3    0    1   

   d_H  d_I  d_M  d_F  d_Y  d_W  d_P  d_S  AA  AR  AN  AD  AC  AQ  AE  AG  AH  \
0    0    2    0    1    0    1    0    1   0   0   0   0   0   0   0   0   0   
1    0    2    0    1    0    1    0    1   0   0   0   0   0   0   0   0   0   
2    0    1    0    1    0    1    0    2   0   0   0   0   0   0   0   0   0   

   AI  AL  AK  AM  AF  AP  AS  AT  AW  AY

In [8]:
print(features['d_KK'])

0    1
1    1
2    1
Name: d_KK, dtype: int64


In [9]:
probas=[class_proba[1] for class_proba in ep.predict_proba(features)]
print(probas)

[np.float64(0.67), np.float64(0.67), np.float64(0.76)]


In [10]:
max_value = max(probas)
max_index = probas.index(max_value)
print(max_index,max_value)
print(seq[max_index:max_index+8])

2 0.76
EAWRIFGS


In [11]:
ep.predict_proba(D.kmer_features_of_protein('IVIDEAWRIFGSDK',8))

array([[0.46, 0.54],
       [0.48, 0.52],
       [0.43, 0.57],
       [0.33, 0.67],
       [0.24, 0.76],
       [0.35, 0.65],
       [0.34, 0.66]])